<h1><b>Data Preprocessing & Cleaning</h1></b>

<h2>Contents</h2>

1. Data Cleaning
2. Lemmatization
3. Stemming
4. Tokenization

Source Code:
https://techcommunity.microsoft.com/t5/educator-developer-blog/how-to-scrape-twitter-data-for-sentiment-analysis-with-python/ba-p/3593365

To conduct data cleaning ONLY, click [here](#Section1)

In [2]:
#Download nltk packages

import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('omw-1.4')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\LENOVO\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\LENOVO\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\LENOVO\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\LENOVO\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [ ]:
%%time

from nltk.corpus import stopwords
 
nltk.download('stopwords')
print(stopwords.words('english'))

In [3]:
#Import libraries

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import nltk
import string
import re
import contractions

#to detect text language
from langdetect import detect, LangDetectException

#stopwords
from nltk.corpus import stopwords

#Tokenization
from nltk.tokenize import word_tokenize

#Stemming
from nltk.stem import PorterStemmer

#Lemmatizing
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

In [4]:
# Load csv file containing tweets dataset
def load_data():
    data = pd.read_csv(r"C:\Users\LENOVO\Documents\Degree Life\FYP Journey\Dataset\Python Scraping\Snscrape\Feb 2022 - Mar 2023\Hungry OR Hunger\States\Hungry OR Hunger_Kuala Lumpur [Coordinates] - Snscrape.csv")
    return data

tweet_df = load_data()
display(tweet_df)

print('Number of original rows:', len(tweet_df))

,Datetime,ID,Username,Text,Location
0,2023-03-30 23:41:11+00:00,1641586401568509952,NumanAfifi,@ashvinmenon Turns out they’re the best when t...,Kuala Lumpur
1,2023-03-30 12:07:36+00:00,1641411855225851905,marcosrahman,this week is super hectic. literally the harde...,"Kuala Lumpur, Malaysia"
2,2023-03-30 10:12:34+00:00,1641382904185110530,LynnTweetss,Hungry or annoyed or just grumpy?,Kuala Lumpur City
3,2023-03-30 06:00:53+00:00,1641319565018959872,lzmnilz,sukiya!! tomorrow!! cepat!! hungry!!,Kuala Lumpur City
4,2023-03-30 04:00:54+00:00,1641289370824232960,Hemarubeny30,So hungry 😩,"kajang,Malaysia"
...,...,...,...,...,...
960,2022-03-06 04:25:51+00:00,1500326770817773569,ffarhan26,@zulhilmi_imi Ah to ease your hunger\n✨at leas...,Malaysia
961,2022-02-26 15:19:57+00:00,1497592277778452481,sarahshdn,@cygaraga I may add some since I've been there...,"Kuala Lumpur, Malaysia"
962,2022-02-10 14:08:20+00:00,1491776048253390848,al_seng,"@AzalinaOthmanS @MYParlimen To me, for a count...",NaN
963,2022-02-09 08:00:03+00:00,1491320979880288256,afmnurdin,dia mcm hunger game or maze runner show dah. S...,NaN


Number of original rows: 965


In [5]:
#Drop unnecessary columns (ID)
new_df = tweet_df.drop(['ID'], axis=1)
display(new_df)

,Datetime,Username,Text,Location
0,2023-03-30 23:41:11+00:00,NumanAfifi,@ashvinmenon Turns out they’re the best when t...,Kuala Lumpur
1,2023-03-30 12:07:36+00:00,marcosrahman,this week is super hectic. literally the harde...,"Kuala Lumpur, Malaysia"
2,2023-03-30 10:12:34+00:00,LynnTweetss,Hungry or annoyed or just grumpy?,Kuala Lumpur City
3,2023-03-30 06:00:53+00:00,lzmnilz,sukiya!! tomorrow!! cepat!! hungry!!,Kuala Lumpur City
4,2023-03-30 04:00:54+00:00,Hemarubeny30,So hungry 😩,"kajang,Malaysia"
...,...,...,...,...
960,2022-03-06 04:25:51+00:00,ffarhan26,@zulhilmi_imi Ah to ease your hunger\n✨at leas...,Malaysia
961,2022-02-26 15:19:57+00:00,sarahshdn,@cygaraga I may add some since I've been there...,"Kuala Lumpur, Malaysia"
962,2022-02-10 14:08:20+00:00,al_seng,"@AzalinaOthmanS @MYParlimen To me, for a count...",NaN
963,2022-02-09 08:00:03+00:00,afmnurdin,dia mcm hunger game or maze runner show dah. S...,NaN


<h1><b>Data Preprocessing</h1></b>

In [6]:
stopwords = nltk.corpus.stopwords.words('english')

In [7]:
def expand_contractions(text):
    # Expand contractions
    expanded_text = contractions.fix(text)
    return expanded_text

#clean the tweets with a function
def cleanedTweets(text):
    # Expand contractions
    text = expand_contractions(text)
    #lowercase text
    text = text.lower()
    # remove @ and links
    text = ' '.join(re.sub("(@[A-Za-z0-9_]+|[^0-9A-Za-z \t]) | (\w+:\/\/\S+)"," ",text).split())
    #Remove repeating words
    text = re.sub(r'\@\w+|\#\w+|\d+','', text)
    #Remove punctuation & numbers
    punct = str.maketrans('','',string.punctuation)
    text = text.translate(punct)
    #Removing all URLS (hhtps and www)
    text = re.sub(r'http\S+','', text)
    text = re.sub(r'www\S+','', text)
    # remove special characters
    text = re.sub(r'[^\x00-\x7F]+','', text)
    #Tokenizing words & removing stop words from the tweets
    tokens = word_tokenize(text)
    filtered_words = [w for w in tokens if w not in stopwords]
    #Lemmatizing words
    lemmatizer = WordNetLemmatizer()
    lemma_words = [lemmatizer.lemmatize(w) for w in filtered_words]
    text = " ".join(lemma_words)
    #Stemming words
    stemmer = PorterStemmer()
    stem_words = [stemmer.stem(w) for w in lemma_words]
    
    #Return tweets based on different phases
    return {'text': text, 'tokens': tokens, 
            'filtered_words': filtered_words, 
            'lemma_words': lemma_words, 
            'stem_words': stem_words}

In [9]:
# Apply the cleanedTweets function to the 'Text' column and create new columns for each phase
new_df[['Cleaned_Text', 'Tokens', 'Filtered_Words', 'Lemmatized_Words', 'Stemmed_Words']] = new_df['Text'].apply(cleanedTweets).apply(pd.Series)
display(new_df)

,Datetime,Username,Text,Location,Cleaned_Tweets,Cleaned_Text,Tokens,Filtered_Words,Lemmatized_Words,Stemmed_Words
0,2023-03-30 23:41:11+00:00,NumanAfifi,@ashvinmenon Turns out they’re the best when t...,Kuala Lumpur,"{'text': 'turn best hungry', 'tokens': ['turns...",turn best hungry,"[turns, out, they, are, the, best, when, they,...","[turns, best, hungry]","[turn, best, hungry]","[turn, best, hungri]"
1,2023-03-30 12:07:36+00:00,marcosrahman,this week is super hectic. literally the harde...,"Kuala Lumpur, Malaysia",{'text': 'week super hectic literally hardest ...,week super hectic literally hardest week work ...,"[this, week, is, super, hectic, literally, the...","[week, super, hectic, literally, hardest, week...","[week, super, hectic, literally, hardest, week...","[week, super, hectic, liter, hardest, week, wo..."
2,2023-03-30 10:12:34+00:00,LynnTweetss,Hungry or annoyed or just grumpy?,Kuala Lumpur City,"{'text': 'hungry annoyed grumpy', 'tokens': ['...",hungry annoyed grumpy,"[hungry, or, annoyed, or, just, grumpy]","[hungry, annoyed, grumpy]","[hungry, annoyed, grumpy]","[hungri, annoy, grumpi]"
3,2023-03-30 06:00:53+00:00,lzmnilz,sukiya!! tomorrow!! cepat!! hungry!!,Kuala Lumpur City,"{'text': 'sukiya tomorrow cepat hungry', 'toke...",sukiya tomorrow cepat hungry,"[sukiya, tomorrow, cepat, hungry]","[sukiya, tomorrow, cepat, hungry]","[sukiya, tomorrow, cepat, hungry]","[sukiya, tomorrow, cepat, hungri]"
4,2023-03-30 04:00:54+00:00,Hemarubeny30,So hungry 😩,"kajang,Malaysia","{'text': 'hungry', 'tokens': ['so', 'hungry'],...",hungry,"[so, hungry]",[hungry],[hungry],[hungri]
...,...,...,...,...,...,...,...,...,...,...
960,2022-03-06 04:25:51+00:00,ffarhan26,@zulhilmi_imi Ah to ease your hunger\n✨at leas...,Malaysia,"{'text': 'ah ease hunger least', 'tokens': ['a...",ah ease hunger least,"[ah, to, ease, your, hunger, at, least]","[ah, ease, hunger, least]","[ah, ease, hunger, least]","[ah, eas, hunger, least]"
961,2022-02-26 15:19:57+00:00,sarahshdn,@cygaraga I may add some since I've been there...,"Kuala Lumpur, Malaysia",{'text': 'may add since got kicked track covid...,may add since got kicked track covid change ma...,"[i, may, add, some, since, i, have, been, ther...","[may, add, since, got, kicked, track, covid, c...","[may, add, since, got, kicked, track, covid, c...","[may, add, sinc, got, kick, track, covid, chan..."
962,2022-02-10 14:08:20+00:00,al_seng,"@AzalinaOthmanS @MYParlimen To me, for a count...",NaN,{'text': 'country country embrace democracy la...,country country embrace democracy law guide po...,"[to, me, for, a, country, a, country, which, e...","[country, country, embrace, democracy, law, gu...","[country, country, embrace, democracy, law, gu...","[countri, countri, embrac, democraci, law, gui..."
963,2022-02-09 08:00:03+00:00,afmnurdin,dia mcm hunger game or maze runner show dah. S...,NaN,{'text': 'dia mcm hunger game maze runner show...,dia mcm hunger game maze runner show dah siapa...,"[dia, mcm, hunger, game, or, maze, runner, sho...","[dia, mcm, hunger, game, maze, runner, show, d...","[dia, mcm, hunger, game, maze, runner, show, d...","[dia, mcm, hunger, game, maze, runner, show, d..."


In [8]:
#Create a new column called "Cleaned_Tweets" by applying preprocessed tweets function to the 'Tweet' column
new_df['Cleaned_Tweets'] = new_df['Text'].apply(lambda x: cleanedTweets(x))
display(new_df)

,Datetime,Username,Text,Location,Cleaned_Tweets
0,2023-03-30 23:41:11+00:00,NumanAfifi,@ashvinmenon Turns out they’re the best when t...,Kuala Lumpur,"{'text': 'turn best hungry', 'tokens': ['turns..."
1,2023-03-30 12:07:36+00:00,marcosrahman,this week is super hectic. literally the harde...,"Kuala Lumpur, Malaysia",{'text': 'week super hectic literally hardest ...
2,2023-03-30 10:12:34+00:00,LynnTweetss,Hungry or annoyed or just grumpy?,Kuala Lumpur City,"{'text': 'hungry annoyed grumpy', 'tokens': ['..."
3,2023-03-30 06:00:53+00:00,lzmnilz,sukiya!! tomorrow!! cepat!! hungry!!,Kuala Lumpur City,"{'text': 'sukiya tomorrow cepat hungry', 'toke..."
4,2023-03-30 04:00:54+00:00,Hemarubeny30,So hungry 😩,"kajang,Malaysia","{'text': 'hungry', 'tokens': ['so', 'hungry'],..."
...,...,...,...,...,...
960,2022-03-06 04:25:51+00:00,ffarhan26,@zulhilmi_imi Ah to ease your hunger\n✨at leas...,Malaysia,"{'text': 'ah ease hunger least', 'tokens': ['a..."
961,2022-02-26 15:19:57+00:00,sarahshdn,@cygaraga I may add some since I've been there...,"Kuala Lumpur, Malaysia",{'text': 'may add since got kicked track covid...
962,2022-02-10 14:08:20+00:00,al_seng,"@AzalinaOthmanS @MYParlimen To me, for a count...",NaN,{'text': 'country country embrace democracy la...
963,2022-02-09 08:00:03+00:00,afmnurdin,dia mcm hunger game or maze runner show dah. S...,NaN,{'text': 'dia mcm hunger game maze runner show...


<h2>Data Cleaning</h2>

Contents:
1. Remove Non-English Tweets
2. Remove Duplicate Tweets

In [ ]:
# Remove non-English tweets
processed_df = new_df[new_df['Cleaned_Tweets'].apply(lambda x: detect(x) == 'en')]
display(processed_df)

print('Number of rows left:', len(processed_df))

In [ ]:
# Define a function to check if text is in English
def is_english(text):
    try:
        lang = detect(text)
        return lang == 'en'
    except LangDetectException:
        return False

# Remove non-English tweets
processed_df = new_df[new_df['Cleaned_Tweets'].apply(is_english)]
display(processed_df)
print('Number of rows left:', len(processed_df))

In [ ]:
#Remove duplicate tweets in the dataset
cleaned_df = processed_df.drop_duplicates(subset='Cleaned_Tweets')
display(cleaned_df)

print('Number of rows left:', len(cleaned_df))

In [ ]:
# Select only the 'Text' and 'Cleaned_Tweets' columns
#List of columns to drop
columns_to_drop = ["Tokens", "Filtered_Words", "Lemmatized_Words", "Stemmed_Words"]

#Drop the irrelavant columns mentioned in the list
final_df = cleaned_df.drop(columns_to_drop, axis=1)
display(final_df)

In [ ]:
#Save the cleaned dataset
cleaned_df.to_csv('CLEANED_Hungry_Malaysia (Coordinates) - Snscrape.csv', index=False)

## Data Cleaning ONLY<a name="Section1"></a>

This process does not include data preprocessing phase (Lemmatizing, Stemming, Tokenization, Removal of stop words)

In [ ]:
#Import libraries

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import nltk
import string
import re
import contractions

#to detect text language
from langdetect import detect, LangDetectException

#stopwords
from nltk.corpus import stopwords

#Tokenization
from nltk.tokenize import word_tokenize

#Stemming
from nltk.stem import PorterStemmer

#Lemmatizing
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

In [ ]:
# Load csv file containing tweets dataset
def load_data():
    data = pd.read_csv(r"C:\Users\LENOVO\Documents\Degree Life\FYP Journey\Dataset\Python Scraping\Snscrape\Feb 2022 - Mar 2023\Hungry\States\[COMBINED] Hungry_States (Coordinates) - Snscrape.csv")
    return data

tweet_df = load_data()
display(tweet_df)

print('Number of original rows:', len(tweet_df))

In [ ]:
#Drop unnecessary columns (ID)
new_df = tweet_df.drop(['ID'], axis=1)
display(new_df)

In [ ]:
#Data Cleaning process STARTS HERE

def expand_contractions(text):
    # Expand contractions
    expanded_text = contractions.fix(text)
    return expanded_text

#clean the tweets with a function
def cleanedTweets(text):
    # Expand contractions
    text = expand_contractions(text)
    #lowercase text
    text = text.lower()
    # remove @ and links
    text = ' '.join(re.sub("(@[A-Za-z0-9_]+|[^0-9A-Za-z \t]) | (\w+:\/\/\S+)"," ",text).split())
    #Remove repeating words
    text = re.sub(r'\@\w+|\#\w+|\d+','', text)
    #Remove punctuation & numbers
    punct = str.maketrans('','',string.punctuation)
    text = text.translate(punct)
    #Removing all URLS (hhtps and www)
    text = re.sub(r'http\S+','', text)
    text = re.sub(r'www\S+','', text)
    # remove special characters
    text = re.sub(r'[^\x00-\x7F]+','', text)

    return text

In [ ]:
#Create a new column called "Cleaned_Tweets" by applying preprocessed tweets function to the 'Tweet' column
new_df['Cleaned_Tweets'] = new_df['Text'].apply(lambda x: cleanedTweets(x))
display(new_df)

In [ ]:
# Remove non-English tweets
processed_df = new_df[new_df['Cleaned_Tweets'].apply(lambda x: detect(x) == 'en')]
display(processed_df)

print('Number of rows left:', len(processed_df))

In [ ]:
# Define a function to check if text is in English
def is_english(text):
    try:
        lang = detect(text)
        return lang == 'en'
    except LangDetectException:
        return False

# Remove non-English tweets
processed_df = new_df[new_df['Cleaned_Tweets'].apply(is_english)]
display(processed_df)
print('Number of rows left:', len(processed_df))

In [ ]:
#Remove duplicate tweets in the dataset
cleaned_df = processed_df.drop_duplicates(subset='Cleaned_Tweets')
display(cleaned_df)

print('Number of rows left:', len(cleaned_df))

In [ ]:
#Save the cleaned dataset
cleaned_df.to_csv('CLEANED_Hungry_Malaysia (Coordinates) - Snscrape.csv', index=False)